En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
file_path = "/Users/blanc/Documents/challenge_DE/data/farmers-protest-tweets-2021-2-4.json"

# Primer desafío

## Introducción y estrategia

El objetivo de esta tarea es obtener las diez fechas con más twits y sus respectivos usuarios con mayor cantidad de twits en ellas. 

Mi primer enfoque en esta tarea fue pensar en algún paquete que fuera óptimo para procesar datos y por defecto descarté `pandas`, ya que a pesar de ser muy cómodo y útil para el análisis de datos, no se destaca por su eficiencia. Por tanto mi primera opción fue `polars`. Mi primer código se veía más o menos así:

```
df_scan = pl.scan_ndjson(file_path, infer_schema_length=None)

df_output = df_scan \
    .select(
        [
            pl.col('date').str.strptime(pl.Datetime).dt.date().alias('datetime'),
            pl.col('user').struct.field('username').alias('username')
        ]
    ) \
    .group_by(['datetime', 'username']) \
    .agg(
        twits_count = pl.count('username')
    ) \
    .group_by('datetime') \
    .agg([
        pl.col('username').gather(pl.col('twits_count').arg_max()),
        pl.col('twits_count').gather(pl.col('twits_count').arg_max())
    ]) \
    .with_columns(
        pl.col('username').list.first(),
        pl.col('twits_count').list.first()
    ) \
    .sort('twits_count', descending=True) \
    .select(['datetime', 'username']) \
    .head(10)
```

Tan pronto comprobé que estas líneas cumplían el objetivo, me dispuse a buscar otro método para optimizar el uso de memoria, ya que `polars` utiliza procesamiento en paralelo y si bien seguramente era muy rápido el procesamiento, exigía mucho en términos de memoria. Me imaginé que un enfoque secuencial, recorriendo el archivo json línea a línea sería más eficiente aunque quizás más lento. Sin embargo, pronto comprobé que este método no solo era mejor en términos de uso de memoria, sino también de velocidad, utilizando tan solo un 33% del tiempo que `polars` requería para completar la tarea. Es entonces que llegué a la función `q1_memory`.

La estructura de la función es la siguiente:

1. Genera una instancia de contador, utilizando la clase de python Counter
2. Recorre línea por línea el archivo json almacenando en el contador la cantidad de menciones para cada fecha-usuario.
3. Mediante la función `most_common`, obtenemos el usuario más repetido por fecha, y lo guardamos en un array `results`.
4. Ordenamos el array `results` de forma descendiente por la "columna" `count` y limitados el resultado a 10 elementos. 
5. Finalmente, retornamos el resultado.

```
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    user_counts_per_date = defaultdict(Counter)

    with open(file_path, 'r') as f:
        for line in f:
            record = json.loads(line)
            date = datetime.strptime(record['date'], '%Y-%m-%dT%H:%M:%S%z').date()
            username = record['user']['username']
            user_counts_per_date[date][username] += 1

    results = []
    for date, counter in user_counts_per_date.items():
        most_common_user, count = counter.most_common(1)[0]
        results.append((date, most_common_user, count))

    top_results = sorted(results, key=lambda x: (-x[2], x[0]))[:10]
    return [(date, username) for date, username, _ in top_results]
```

### Rendimiento de `q1_memory`

Para analizar el rendimiento de las funciones, se les ha colocado el decorador `@track_execution_time`, que es una función auxiliar que busca trackear el tiempo de ejecución de cada una. Además, sumaremos el uso de `memory_profiler` para monitorear el consumo de memoria.

In [2]:
%load_ext memory_profiler

In [3]:
# Importación de función q1_memory
from q1_memory import q1_memory

In [5]:
%%memit # Usamos este magic command para aplicar el memory_profiler en esta celda

results_q1_memory = q1_memory(file_path)

q1_memory executed in 2.4085888862609863 seconds.
peak memory: 72.28 MiB, increment: 3.83 MiB


Como puede apreciarse, el tiempo de ejecución es de 2.4 segundos, generando un incremento de 3.8 megas de ram, ubicándonos en un pico de 72.3 en total. A continuación, los resultados.

In [6]:
results_q1_memory

[(datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 21), 'Surrypuria'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist')]

### Rendimiento de `q1_time`

Tal como mencioné anteriormente, era tal la diferencia tanto en uso de memoria como en velocidad de ejecución entre el enfoque secuencial de json y el uso de `polars`, que me hizo cuestionarme el utilizar `polars` como tal. Entendí que la fortaleza de `polars` radicaba en el uso de procesamiento paralelo, pero al parecer esto no llegaba a justificar su uso. Por tanto me propuse mantener la estructura de `q1_memory` pero agregándole procesamiento paralelo, buscando obtener el mismo beneficio que `polars` pero sin tener que utilizar tan pesada librería. Es así que di con el paquete `concurrency`, que me permitió desarrollar la misma estrategía que `q1_memory` pero de forma paralela.

La estructura de la función es la siguiente:
1. Definimos `process_lines`, que básicamente realiza el trabajo de contar lineas por fecha-usuario, tal como en la otra función.
2. Generamos una serie de chunks sobre el dataset original, de modo de procesar esta información paralelamente más adelante.
3. Generamos una instancia de `ThreadPoolExecutor`, en donde se aplicará la función `process_lines` a cada chunk creado
4. En la medida que los chunks se van procesando, se agrupan en el array `results`
5. Mediante la función `most_common`, obtenemos el usuario más repetido por fecha, y lo guardamos en un array `final_results`.
6. Ordenamos el array `results` de forma descendiente por la "columna" `count` y limitados el resultado a 10 elementos. 
7. Finalmente, retornamos el resultado.

Como se habrá notado, la mayoría de los pasos son exactamente igual que en la función q1_memory, con una única adición de procesamiento paralelo para mayor velocidad.


```
def process_lines(lines):
    local_counts = defaultdict(Counter)
    for line in lines:
        record = json.loads(line)
        date = datetime.strptime(record['date'], '%Y-%m-%dT%H:%M:%S%z').date()
        username = record['user']['username']
        local_counts[date][username] += 1
    return local_counts

@track_execution_time
def q1_time(file_path: str, lines_per_chunk = 1000, num_workers = 4) -> List[Tuple[datetime.date, str]]:
    
    chunks = []

    with open(file_path, 'r') as f:
        chunk = []
        for line in f:
            chunk.append(line)
            if len(chunk) >= lines_per_chunk:
                chunks.append(chunk)
                chunk = []
        if chunk:
            chunks.append(chunk)

    results = defaultdict(Counter)
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        futures = [executor.submit(process_lines, chunk) for chunk in chunks]
        for future in as_completed(futures):
            local_counts = future.result()
            for date, counter in local_counts.items():
                results[date] += counter

    final_results = []
    for date, counter in results.items():
        most_common_user, count = counter.most_common(1)[0]
        final_results.append((date, most_common_user, count))

    top_results = sorted(final_results, key=lambda x: (-x[2], x[0]))[:10]
    return [(date, username) for date, username, _ in top_results]
```

In [7]:
# Importación de función q1_time
from q1_time import q1_time

In [8]:
%%memit

results_q1_time = q1_time(file_path)

q1_time executed in 2.806195020675659 seconds.
peak memory: 503.52 MiB, increment: 442.39 MiB


A pesar de que teóricamente el objetivo de utilizar este método es optimizar el tiempo de ejecución, ese objetivo no se cumple en este caso. Mi hipótesis para este resultado es que el tamaño del dataset no permite brillar la utilidad del procesamiento paralelo. En la medida que esta tarea se aplique a un dataset de mayor tamaño, es donde el enfoque secuencial comenzará a quedarse atrás en términos de tiempo de ejecución en comparación al enfoque paralelo.

In [10]:
results_q1_time

[(datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 21), 'Surrypuria'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist')]